In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from itertools import islice
import torch
import os
import wandb
# from sklearn.preprocessing import StandardScaler
import pandas as pd
from gluonts.dataset.repository import get_dataset, dataset_names
from gluonts.dataset.util import to_pandas
from gluonts.dataset.common import ListDataset
from tqdm.autonotebook import tqdm
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.pandas import PandasDataset
from lag_llama.gluon.estimator import LagLlamaEstimator

/tmp/ipykernel_19922/4162833500.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


sensor (2,3,4) (20,21)highly correlated

sensor 6 jumps between two values

sensor 7,8,9 

sensor 11,12,13,14,15,17

# in total 15 sensors still relevant for train_001 and train_003
# for train_002, train_004 all sensor signals are relevant

In [2]:
keys = ['cycle', 'op_setting_1', 'op_setting_2', 'op_setting_3'] + [f'sensor_{i}' for i in range(1, 24)]

def df_preprocessing(list, col):
    temp = pd.DataFrame()
    for dataset in list:
        df = pd.read_csv('cmapss_data/'+dataset+'.txt', index_col=0, sep=" ", header=None)
        df.columns = keys
        df.dropna(axis=1, inplace=True)
        df.index.names = ['engine']
        df_col = df[['cycle', col]]
        df_col = df_col.reset_index()
        pivot = df_col.pivot(index='cycle', columns='engine', values=col)
        temp = pd.concat([temp, pivot], axis=1)
    temp.columns = range(temp.shape[1])
    temp.index = pd.to_datetime(temp.index, origin='2000-01-01', unit='D')
    for col in temp.columns:
        temp[col] = temp[col].shift(periods=temp[col].isna().sum(), fill_value=np.nan)
    return temp

In [ ]:
df = df_preprocessing(['train_FD001', 'train_FD003'], 'sensor_14') # 200 trajectories
df.dropna(axis=0, inplace=True)
columns = df.columns.tolist()

np.random.seed(2)
np.random.shuffle(columns) # reproduceably shuffle

split_index = int(0.70 * len(columns))
df = (df - df.mean()) / df.std() # standardization

In [ ]:
train_df = df[columns[:split_index]] # 140 trajectories, all 128 in length
test_df = df[columns[split_index:]]
#train_df.to_csv(f'train_df.csv')
#test_df.to_csv(f'test_df.csv')